In [1]:
import matplotlib
import os
import sys
matplotlib.use('TkAgg')
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import average_precision_score
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
from importlib import reload
sys.path.append(os.path.abspath(".."))
import src.data_processing.data_augmentation as daug
import src.models.CNN as CNN
reload(CNN)
from src.models.active_learning_kcluster import kCenterGreedy

2025-09-07 12:02:09.333684: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Read in data
df = pd.read_csv("/Users/jameshill/PycharmProjects/bioacoustic-classifier/src/data/annotations/spectrogram_labels.csv")
df['filepath'] = "/Users/jameshill/PycharmProjects/bioacoustic-classifier/data/processed/spectrogram_3s/" + df['filename'] + ".png"
df['split_labels'] = df['label'].str.split('_and_')

In [3]:
# Get arrays
filepaths = df['filepath'].values
# Initialise and fit multi-label encoder
mle = MultiLabelBinarizer()
multi_labels = mle.fit_transform(df['split_labels'])
labels = multi_labels
cnn = CNN.define_cnn(mle.classes_)
assert cnn.output_shape[-1] == len(mle.classes_)

In [7]:
mle.classes_

array(['background_noise', 'blackcap', 'bluetit', 'carrion_crow',
       'chiffchaff', 'common_whitethroat', 'dog_barking', 'dunnock',
       'eurasian_skylar', 'eurasian_skylark', 'pheasant', 'unknown_bird',
       'unknown_bird_10', 'unknown_bird_11_12', 'unknown_bird_12',
       'unknown_bird_12_13', 'unknown_bird_13', 'unknown_bird_14',
       'unknown_bird_15', 'unknown_bird_15_13', 'unknown_bird_16',
       'unknown_bird_17', 'unknown_bird_18', 'unknown_bird_2',
       'unknown_bird_3', 'unknown_bird_4', 'unknown_bird_5',
       'unknown_bird_9', 'woodpigeon', 'wren', 'yellowhammer',
       'yellowhammer_blackcap'], dtype=object)

In [ ]:
def conv_encoder(input_shape=(64,512,1)):
    inp = keras.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, padding="same", activation="relu")(inp)
    x = layers.MaxPooling2D(2)(x)             # 32x256
    x = layers.Conv2D(64, 3, padding="same", activation="relu")(x)
    x = layers.MaxPooling2D(2)(x)             # 16x128
    x = layers.Conv2D(128, 3, padding="same", activation="relu")(x)
    x = layers.MaxPooling2D(2)(x)             # 8x64
    x = layers.Conv2D(256, 3, padding="same", activation="relu",
                      kernel_regularizer=keras.regularizers.l2(1e-4))(x)
    # Bottleneck is a **feature map** (8x64x256), not GAP
    return keras.Model(inp, x, name="conv_encoder")

In [ ]:
def build_cae(input_shape=(64,512,1)):
    enc = conv_encoder(input_shape)
    z   = enc.output                     # (8,64,256)

    x = layers.UpSampling2D((2,2))(z)    # 16x128
    x = layers.Conv2D(128, 3, padding="same", activation="relu")(x)
    x = layers.UpSampling2D((2,2))(x)    # 32x256
    x = layers.Conv2D(64, 3, padding="same", activation="relu")(x)
    x = layers.UpSampling2D((2,2))(x)    # 64x512
    x = layers.Conv2D(32, 3, padding="same", activation="relu")(x)
    out = layers.Conv2D(1, 3, padding="same", activation="sigmoid")(x)

    cae = keras.Model(enc.input, out, name="cae")
    return cae, enc
